# Imports

In [2]:
import numpy as np
import pandas as pd
import os

import torch
from torch import cuda
from torchvision import models
import torch.nn as nn
from torchsummary import summary
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import random_split

# !wget -c https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
import helper

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook

# CUDA and GPU check

In [3]:
!module add u18/cuda/10.0
!module add u18/cudnn/7.6-cuda-10.0

train_on_gpu = cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')

if train_on_gpu:
    gpu_count = cuda.device_count()
    print(f'{gpu_count} gpus detected.')
    if gpu_count > 1:
        multi_gpu = True
    else:
        multi_gpu = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Train on gpu: True
4 gpus detected.


device(type='cuda')

# Data loaders

In [4]:
event_data_dir = "/scratch/adithyasunil/event"
categories = os.listdir(event_data_dir)
categories

['Stridor',
 'Normal',
 'Wheeze',
 'Wheeze+Crackle',
 'Rhonchi',
 'Fine Crackle',
 'Coarse Crackle']

In [5]:
batch_size=256

transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])
dataset = datasets.ImageFolder(event_data_dir, transform=transform)
# print(len(dataset))
train_dataset, test_dataset = random_split(dataset, [31000, 1622])
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

In [6]:
train_data_iter = iter(train_dataloader)

fig, axes = plt.subplots(figsize=(10,10), nrows=8, ncols=4)
for row in range(8):
    for col in range(4):
        images, labels = next(train_data_iter)
        ax = axes[row, col]
        helper.imshow(images[0], ax=ax, normalize=False)
        ax.title.set_text(dataset.classes[labels[0]])
#         print(labels[0])
#         print(dataset.classes[labels[0]])

Error in callback <function flush_figures at 0x14b0a8a439e0> (for post_execute):


KeyboardInterrupt: 

# CNN models

In [7]:
def get_pretrained_model(model_name, n_classes):
    """
    Used to retrieve pre trained VGG16 or Resnet50 with custom classifier
    """

    if model_name == 'vgg16':
        model = models.vgg16(pretrained=True)

        for param in model.parameters():
            param.requires_grad = False
        n_inputs = model.classifier[0].in_features

        model.classifier = nn.Sequential(
            nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(256, n_classes), nn.LogSoftmax(dim=1))
        
        print("Classifier:")
        print(model.classifier)

    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=True)

        for param in model.parameters():
            param.requires_grad = False

        n_inputs = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(256, n_classes), nn.LogSoftmax(dim=1))
        
        print("Classifier:")
        print(model.fc)
    
    if multi_gpu:
        model = nn.DataParallel(model)
    if train_on_gpu:
        model = model.to('cuda')

    return model

In [8]:
vgg16 = get_pretrained_model('vgg16', 2)

criterion = nn.NLLLoss()
if multi_gpu:
    optimizer = optim.Adam(vgg16.module.classifier.parameters(), lr=0.003)
elif train_on_gpu:
    optimizer = optim.Adam(vgg16.classifier.parameters(), lr=0.003)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

Classifier:
Sequential(
  (0): Linear(in_features=25088, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=256, out_features=2, bias=True)
  (4): LogSoftmax(dim=1)
)


In [9]:
resnet50 = get_pretrained_model('resnet50', 2)

criterion = nn.NLLLoss()
if multi_gpu:
    optimizer = optim.Adam(resnet50.module.fc.parameters(), lr=0.003)
elif train_on_gpu:
    optimizer = optim.Adam(resnet50.classifier.parameters(), lr=0.003)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

Classifier:
Sequential(
  (0): Linear(in_features=2048, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=256, out_features=2, bias=True)
  (4): LogSoftmax(dim=1)
)


In [10]:
n_classes = len(categories)
model = get_pretrained_model('vgg16', n_classes)

if multi_gpu:
    summary(
        model.module,
        input_size=(3, 224, 224),
        batch_size=batch_size,
        device='cuda')
else:
    summary(
        model, 
        input_size=(3, 224, 224), 
        batch_size=batch_size, 
        device='cuda')

total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

criterion = nn.CrossEntropyLoss()
criterion.cuda()
if multi_gpu:
    optimizer = optim.Adam(model.module.classifier.parameters(), lr=0.003)
elif train_on_gpu:
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

Classifier:
Sequential(
  (0): Linear(in_features=25088, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=256, out_features=7, bias=True)
  (4): LogSoftmax(dim=1)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [256, 64, 224, 224]           1,792
              ReLU-2        [256, 64, 224, 224]               0
            Conv2d-3        [256, 64, 224, 224]          36,928
              ReLU-4        [256, 64, 224, 224]               0
         MaxPool2d-5        [256, 64, 112, 112]               0
            Conv2d-6       [256, 128, 112, 112]          73,856
              ReLU-7       [256, 128, 112, 112]               0
            Conv2d-8       [256, 128, 112, 112]         147,584
              ReLU-9       [256, 128, 112, 112]               0
        MaxPool2d-10         [256, 128, 56, 56]               0
         

# Training

In [ ]:
save_path = "/home2/adithyasunil/resp_sound/model_checkpoints/"

for epoch in tqdm_notebook(range(100)):  # loop over the dataset multiple times
#     print("Epoch ",epoch+1)
    running_loss = 0.0
    for i, data in enumerate(tqdm_notebook(train_dataloader), 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
#         if epoch%10==0:
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        running_loss = 0.0
    if (epoch+1)%5==0:
        chkp_save_path = save_path + "model_event_epoch_" + str(epoch+1) + ".pt" 
        torch.save(model.state_dict(), chkp_save_path)
print('Finished Training')

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

...

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

In [12]:
chkp_epoch_to_load = 30
model.load_state_dict(torch.load(f'/home2/adithyasunil/resp_sound/model_checkpoints/model_event_epoch_{chkp_epoch_to_load}.pt'))

error = 0
# p = 0
for l, data in tqdm_notebook(enumerate(test_dataloader, 0)):
#     print(p)
    inputs, labels = data
#     print(labels)
    optimizer.zero_grad()
    outputs = model(inputs.to(device))
    output = np.zeros(outputs.cpu().shape)
    for i in range(len(outputs)):
        for j in range(len(outputs[0])):
            k = outputs[i][j].cpu()
            output[i][j] = k
#     for i in range(len(output)):
#         print(list(output[i]).index(max(output[0])))
#     print("____________")
    if labels.cpu()!=list(output[i]).index(max(output[0])):
        error = error + 1
#     p = p+1

0it [00:00, ?it/s]

In [14]:
accuracy = 100*(1-error/len(test_dataloader.dataset))
print(accuracy)

98.27373612823675
